# Adding/removing particles in GCMC

This script uses MC moves for adding and removing particles in the Grand-canonical ensemble.    
    
This simulation models a lattice-gas, using the chemical potential of each species to define the acceptance criteria.    
    
For more information, see <i> Understanding molecular simulations (Frenkel & Smit), Chapter 5.6 </i>

## Imports

In [ ]:
# standard library imports
from math import ceil, exp, floor, sqrt, pi
from random import random, randint

# related third party imports
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

# local application/library specific imports
from utilities import draw_lattice_configuration, Histogram

%matplotlib inline

## Simulation parameters

In [ ]:
# simulation parameters
length                   = 10                # number of sites per side
sites                    = length * length   # total number of sites
particles                = 10                # number of particles
cycles                   = 10000             # number of MC cycles
add_remove_probability   = 0.5               # probability of add/remove move
drawing_frequency        = 1000              # draw configuration every

## Initializing simulation

In [ ]:
occupancy = np.zeros([length, length])
coordinates = np.zeros([2, particles])

placed_particles = 0
for x in range(length):
    for y in range(length):
        if placed_particles < particles:
            coordinates[0, placed_particles] = x
            coordinates[1, placed_particles] = y
            occupancy[x, y] = 1
            placed_particles += 1
            
draw_lattice_configuration(coordinates, length)

## Create histogram for particle concentration(s)

In [ ]:
value_range        = [0, 1]    # (min, max) particle conc. [frac.]
increment          = 0.025     # bin-width
output_frequency   = 1000      # cycles-per-calculation

histogram = Histogram(value_range, increment, output_frequency)

# set parameters for particle concentration(s), phi
sample_start = 1000            # step to start sampling phi
average_phi = 0.                # holds average phi
sample_count = 0               # holds number of times sampled

## MC simulation

In [ ]:
add_count = 0
remove_count = 0

for cycle in range(cycles):
#    print('%s\t%s' % (cycle, particles))
    if random() < add_remove_probability:
        if random() > 0.5:    # ADDITION MOVE
            x = randint(0, length - 1)
            y = randint(0, length - 1)
            if occupancy[x, y] == 0:
                particles += 1
                coordinates = np.c_[coordinates, [[x], [y]]]
                occupancy[x, y] = 1
                add_count += 1
        else:                 # REMOVAL MOVE
            if random() < 0.05 * particles / float(sites):
                # probability of removal is proportional to phi
                particles -= 1
                particle = randint(0, particles - 1)
                occupancy[coordinates[0, particle], coordinates[1, particle]] = 0
                np.delete(coordinates, np.s_[particle], axis=1)
                remove_count += 1
                
    if particles > 0:  # DISPLACEMENT MOVE
        for sub_cycle in range(sites):
            particle = randint(0, particles - 1)    # choose random particle
            x = randint(0, length - 1)              # choose random x-position
            y = randint(0, length - 1)              # choose random y-position
            if occupancy[x, y] == 0:                # empty old cell
                occupancy[coordinates[0, particle], coordinates[1, particle]] = 0
                coordinates[0, particle] = x        # save the new position
                coordinates[1, particle] = y
                occupancy[x, y] = 1                 # update occupancy
                
    if cycle % drawing_frequency == 0:              # draw configuration
        print('cycle:\t%s' % cycle)
        draw_lattice_configuration(coordinates, length)
        
    if cycle > sample_start:                        # sample average phi
        average_phi += particles / float(sites)
        sample_count += 1
        histogram.add_data(particles / float(sites))

## Analyse results

In [ ]:
print('average phi:\t%s' % (average_phi / sample_count))

h = histogram
simulated = h.histogram / (h.count * h.increment)   # normalize
plt.bar(h.values, simulated, width=h.increment)

# plot theoretical dist.--gaussian with mean=0.5 and variance=0.25
theoretical = [exp(-2 * sites * (i - 0.5) ** 2) * sqrt(2 * sites / pi)
               for i in h.values
              ]
plt.plot(h.values, theoretical, 'r')
plt.show()